In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch.utils.data import DataLoader,TensorDataset

from preprocess import *
from attacks import *
from models import *
from utils import *
from generate_attacks import *

device = 'cuda'
torch.cuda.set_device(2)

In [2]:
train_set = datasets.MNIST(root = '/home/aminul/data',train = True, 
                           transform = transforms.ToTensor(), download = False)
test_set = datasets.MNIST(root = '/home/aminul/data',train = False, 
                          transform = transforms.ToTensor(), download = False)

In [3]:
train_loader = DataLoader(train_set,100,True)
test_loader = DataLoader(test_set,100,False)

train_loader_2 = DataLoader(train_set,1,True)
test_loader_2 = DataLoader(test_set,1,False)

In [4]:
num_classes = 10
learning_rate = 0.01
num_epochs = 5

In [5]:
criterion = nn.CrossEntropyLoss()

model1 = CNN()

opt_1 = torch.optim.SGD(model1.parameters(),lr=learning_rate)

In [6]:
history = train(model1,train_loader,num_epochs,opt_1,criterion,device)
test(model1,test_loader,criterion,device)

  1%|▏         | 8/600 [00:00<00:07, 79.20it/s]

Training: epoch: [1/5]  loss: [1.86] Accuracy [41.44] 


  1%|          | 7/600 [00:00<00:09, 65.55it/s]

Training: epoch: [2/5]  loss: [0.43] Accuracy [87.46] 


  1%|▏         | 8/600 [00:00<00:07, 78.26it/s]

...

Training: epoch: [4/5]  loss: [0.22] Accuracy [93.43] 


 10%|█         | 10/100 [00:00<00:00, 97.72it/s]

Training: epoch: [5/5]  loss: [0.18] Accuracy [94.62] 


100%|██████████| 100/100 [00:01<00:00, 97.44it/s]

Testing: Loss: [0.15] Accuracy [95.52]


In [17]:
def PGD(model,image,labels,eps,attack_steps,attack_lr,criterion,device,random_init = True, target = None, clamp = (0,1)):
    
    model,image,labels = model.to(device),image.to(device),labels.to(device)    
        
    x_adv = image.clone()
    
    if random_init:
        x_adv = x_adv + (torch.rand(image.size(),dtype = image.dtype, device = device) - 0.5) *2 * eps
        
    for i in range(attack_steps):
        
        x_adv.requires_grad = True
        
        model.zero_grad()
        logits = model(x_adv)
        
        if target is None:
            loss = criterion(logits,labels)
            loss.backward()
            grad = x_adv.grad.detach()
            grad = grad.sign()
            x_adv = x_adv + attack_lr * grad
        
        else:
            assert target.size() == y.size()
            loss = F.cross_entropy(logits, target)
            loss.backward()
            grad = x_adv.grad.detach()
            grad = grad.sign()
            x_adv = x_adv - attack_lr * grad
       
        x_adv = image + torch.clamp(x_adv - image, min=-eps, max=eps)
        x_adv = x_adv.detach()
        x_adv = torch.clamp(x_adv, *clamp)
        
        
    return x_adv

In [18]:
def generatePGD(test_loader,model,device,eps,attack_iter,criterion):
    adv_img = []
    l = []
    attack_lr = 0.01
    
    print("Generating PGD Adversarial Images")
    iterator = tqdm(test_loader,ncols=0, leave=False)
    for data,labels in iterator:
        x1 = PGD(model,data,labels,eps,attack_iter,attack_lr,criterion,device)
        l.append(labels)
        x2 = x1.squeeze().cpu().detach().numpy()
        adv_img.append(x2)
            
    x3 = np.array(adv_img)
    x3 = x3.reshape(10000,1,28,28)
    l = np.array(l)
        
    features_test = torch.from_numpy(x3)
    targets_test = torch.from_numpy(l)

    new_dataset = torch.utils.data.TensorDataset(features_test,targets_test)
    new_data_loader = torch.utils.data.DataLoader(new_dataset,100,shuffle = False)
    
    
    return new_data_loader    

In [20]:
eps = 0.1
attack_iter = 20
data_loader = generatePGD(test_loader_2,model1,device,eps,attack_iter,criterion)

  0% 2/10000 [00:00<11:38, 14.31it/s]

Generating PGD Adversarial Images


In [21]:
test(model1,data_loader,criterion,device)

100%|██████████| 100/100 [00:00<00:00, 462.53it/s]

Testing: Loss: [1.20] Accuracy [59.00]
